In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [23]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/shakespeare_data/input.txt

In [24]:
device = torch.device("cpu")
# if torch.backends.mps.is_available() and torch.backends.mps.is_built():
#     device = torch.device("mps")

device

In [25]:
txt = open('shakespeare_data/input.txt', 'r').read()
len(txt)

1115394

In [26]:
txt[0:500]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor"

In [27]:
chars = list(set(txt))
chars.sort()

ctoi = {c:i for i, c in enumerate(chars)}
itoc = {i:c for i, c in enumerate(chars)}
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [28]:
i =  math.floor(0.9 * len(txt))
train_txt = txt[0:i]
valid_txt = txt[i+1:]

len(train_txt), len(valid_txt)

(1003854, 111539)

In [29]:
train_tkns = [ctoi[c] for c in train_txt]
valid_tkns = [ctoi[c] for c in valid_txt]

In [30]:
from numpy.random import randint
block_size = 64

def txt_to_token(t):
    return [ctoi[c] for c in t]


# (B, L)
def random_batch(split="train"):
    data = train_tkns if split == "train" else valid_tkns
    
    i = randint(0, len(data)-block_size-1)
    x = torch.tensor(data[i:i+block_size], device=device)
    y = torch.tensor(data[i+1:i+block_size+1], device=device)
    
    return x, y

x, y = random_batch("train")
x.shape

torch.Size([64])

In [31]:
@torch.no_grad()
def estimate_loss(model,n_iter=10):
    model.eval()
    losses = []

    for split in ["train", "valid"]:   
        loss=0
        for _ in range(n_iter):     
            x, y = random_batch(split)
            logits = model(x) # (L, C)
            #L, C = logits.shape
            loss+= F.cross_entropy(logits, y)
        losses.append(loss.item()/n_iter)

    model.train()
    return losses

In [32]:
@torch.no_grad()
def sample(model):
    model.eval()

    max_len = 500
    tks = [0]*block_size

    for i in range(max_len):
        ctx = torch.tensor(tks[i:i+block_size]) # (L)
        ctx = ctx.view(-1) # (L)

        logits = model(ctx) # (L, C)
        probs = F.softmax(logits, dim=-1) # (L, C)
        probs = probs[-1,:] # (C), # the last in the sequence is the newly generated
        yi = torch.multinomial(probs, 1)
        tks.append(yi.item())

    tks = tks[block_size:]
    chars = [itoc[t] for t in tks]
    model.train()
    return "".join(chars)

In [33]:
import einops

def rotary_encoding(x, base=1e-5, thetas=None):
    """Applies a rotary embedding to a tensor.

    Args:
        x (torch.Tensor): Tensor to apply the rotary embedding to.
            x.shape=(sequence_lenght, n_heads, d_embedding)
        base (float, optional): Base of the logarithm. Defaults to 1e-5.
        thetas (torch.Tensor, optional): Tensor containing the thetas.
            It can be used in case you want to apply learned positional encoding.
            Defaults to None.

    Returns:
        torch.Tensor: Tensor with the rotary embedding applied.
    """

    #pad with zeros if odd, otherwise we cant pair up consecutive elements
    odd = False
    if x.shape[0] % 2 != 0:
        zeros = torch.zeros((1, *x.shape[1:]), device=x.device)
        x = torch.cat([x, zeros], dim=0)
        odd = True

    #pair up consecutive elements
    x1 = einops.rearrange(x, '(n1 n2) ... -> n1 n2 ...', n2=2)

    #pair up elements and swap them
    x2 = x1[:, torch.tensor([1, 0])]
    x2[:, 0] = -x2[:, 0]

    #create phases
    sin, cos = make_sin_cos(x1.shape, base, thetas, device=x.device)

    #apply rotation
    x1 = einops.einsum(x1, cos, 'n ... c, n c -> n ... c')
    x2 = einops.einsum(x2, sin, 'n ... c, n c -> n ... c')
    x = x1+x2
    x = einops.rearrange(x, 'n1 n2 ... -> (n1 n2) ...', n2=2)

    if odd:
        return x[:-1]  # remove padding if odd
    return x


def make_sin_cos(shape, base=1e-5, thetas=None, device='cpu'):

    if thetas is None:
        thetas = torch.logspace(0, 1, shape[-1], base, device=device)
    indices = torch.arange(0, shape[0], device=device)
    phases = einops.einsum(indices, thetas, 'a, c -> a c')

    #rotate
    sin = torch.sin(phases)
    cos = torch.cos(phases)

    #zetas = torch.linspace(gamma,2*x.shape[-1]+gamma,x.shape[-1],device=x.device)/(1+gamma)

    return sin, cos

In [34]:
# return (L, C)
def pos_encoding(x):
    L, C = x.shape
    pos = torch.arange(0, L).view(-1, 1) # (L, 1)
    div = 2 * torch.arange(0, C) / C # (C)
    div = torch.pow(10000, div) # (C)
    e = pos / div
    pe = torch.zeros(L, C)
    pe[:,0::2] = torch.sin(e[:,0::2])
    pe[:,1::2] = torch.cos(e[:,1::2])
    
    pe = pe.to(device)
    return pe

In [35]:
class MultiHeadAttension(nn.Module):    
    
    def __init__(self, head_num, head_size, in_size, out_size, rotary_encoding=False):
        super().__init__()
        
        self.head_size = head_size
        self.head_num = head_num        
        self.attn = nn.Linear(in_size, 3 * head_num * head_size, bias=False)
        self.ffn = nn.Linear(head_num * head_size, out_size, bias=False)
        self.rotary_encoding = rotary_encoding

        
    # x: (L, C)  
    # return: (L, C')
    def forward(self, x):
        L, C = x.shape
        
        z = self.attn(x) # (L, 3 * hn * hs)
        k, q, v = torch.split(z, self.head_num * self.head_size, dim=-1) # (L, hn * hs)

        # reshape the output to have the correct shape
        q = q.view(L, self.head_num, self.head_size)
        k = k.view(L, self.head_num, self.head_size)
        v = v.view(L, self.head_num, self.head_size)

        # apply rotary encoding if needed
        if self.rotary_encoding:
            q = rotary_encoding(q)
            k = rotary_encoding(k)
        
        q=q.permute(1,0,2) # ( hn, L, hs)
        k=k.permute(1,0,2)
        v=v.permute(1,0,2)

        
        q = q.permute(0, 2, 1) # ( hn, hs, L)
        attn = (k @ q) / self.head_size**0.5 # (hn, L, L)
        mask = torch.tril(torch.ones(L, L)) == 0
        mask = mask.to(device)
        attn = attn.masked_fill(mask, -float('inf')) # (B, hn, L, L)
        attn = F.softmax(attn, dim=-1)
        
        y = attn @ v # (hn, L, hs)
        y = y.permute(1, 0, 2) # (L, hn, hs)
        y = y.contiguous().view(L, -1) # (L, hn * hs)
        y = self.ffn(y) # (L, C)
        
        return y 
    
        
x = torch.randn(block_size, 9) # (L, C)
x = x.to(device)
mh = MultiHeadAttension(5, 3, 9, 7)
mh = mh.to(device)
mh(x).shape

torch.Size([64, 7])

In [36]:
class MLP(nn.Module):
    
    def __init__(self, in_size, out_size):
        super().__init__()
        self.linear1 = nn.Linear(in_size, out_size)
        self.linear2 = nn.Linear(out_size, out_size)
    
    # (B, L, C)
    def forward(self, x):
        y = self.linear1(x)
        y = torch.relu(y)
        y = self.linear2(y)
        
        return y

In [37]:
use_rotary=False


class Block(nn.Module):    
    
    def __init__(self, emb_size, head_size):
        super().__init__()
        
        assert emb_size % head_size == 0
        head_num = emb_size // head_size
        
        self.mha = MultiHeadAttension(head_num, 
                                      head_size, 
                                      in_size=emb_size, 
                                      out_size=emb_size,
                                      rotary_encoding=use_rotary)
        self.lnorm1 = nn.LayerNorm(emb_size)
        self.lnorm2 = nn.LayerNorm(emb_size)
        self.ffn = MLP(emb_size, emb_size)
        
        
    # x: (B, L, emb)
    def forward(self, x):
        y = self.mha(x) + x
        y = self.lnorm1(y)
        y = self.ffn(y) + y
        y = self.lnorm2(y)
        return y
    
# x = torch.randn(3, 4, 10)
# b = Block(10, 2)
# b(x)

In [38]:
emb_size = 128
head_size = 32

class Transformer(nn.Module):    
    
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.blocks = nn.Sequential(
            Block(emb_size, head_size),
            Block(emb_size, head_size),
        )
        self.linear = nn.Linear(emb_size, vocab_size)

    # (L) -> (L, C)
    def forward(self, x):
        y = self.embed(x) # (L, emb)
        if not use_rotary:
            y = y + pos_encoding(y) # (L, emb)
        y = self.blocks(y) # (L, emb)
        y = self.linear(y) # (L, vocab)
        
        return y

In [39]:
model = Transformer()
model = model.to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-4)

count = sum([p.numel() for p in model.parameters()])
print(f"total parameter: {count}")

total parameter: 214849


In [40]:
epoch = 80000
eval_interval = 500
eval_size = 500
lossi = []

model.train()

for i in range(epoch):
    optim.zero_grad()

    xb, yb = random_batch()
    logits = model(xb) # (L, C)

    L, C = logits.shape
    loss = F.cross_entropy(logits, yb)
    loss.backward()
    optim.step()

    if i % eval_interval == 0 or i == epoch-1:
        tr, va = estimate_loss(model)
        lossi.append((tr, va))
        print(f"{i:5d}/{epoch}: {tr:.4f}  {va:.4f}")

    0/80000: 4.4375  4.3993
  500/80000: 2.8487  2.7606
 1000/80000: 2.6795  2.6591
 1500/80000: 2.6022  2.5743
 2000/80000: 2.5221  2.6714
 2500/80000: 2.5135  2.5218
 3000/80000: 2.4672  2.4712
 3500/80000: 2.3915  2.3813
 4000/80000: 2.4754  2.3842
 4500/80000: 2.3706  2.3820
 5000/80000: 2.3130  2.4805
 5500/80000: 2.3531  2.3642
 6000/80000: 2.2285  2.2691
 6500/80000: 2.3126  2.3828
 7000/80000: 2.3603  2.4330
 7500/80000: 2.1525  2.2562
 8000/80000: 2.3713  2.2203
 8500/80000: 2.2481  2.2843
 9000/80000: 2.1526  2.2442
 9500/80000: 2.1602  2.1818
10000/80000: 2.2077  2.2019
10500/80000: 2.3275  2.2109
11000/80000: 2.1323  2.2588
11500/80000: 2.1680  2.2122
12000/80000: 2.0331  2.2068
12500/80000: 2.1375  2.1906
13000/80000: 2.1628  2.1230
13500/80000: 2.2217  2.2068
14000/80000: 2.1822  2.1622
14500/80000: 2.1279  2.1909
15000/80000: 2.1791  2.1814
15500/80000: 2.0448  2.1582
16000/80000: 2.0736  2.1378
16500/80000: 2.1359  2.1472
17000/80000: 2.0423  2.1196
17500/80000: 2.0780 

In [43]:
tr_loss, va_loss = estimate_loss(model)

print(f"train: {tr_loss:.4f}")
print(f"valid: {va_loss:.4f}")

train: 1.7554
valid: 1.7840


In [44]:
print(sample(model))

LAUTOLYCU S:
To thilers as patigar:
Leake an and three; beggaren'roy, and firttione.

ChORTER:
Wethen leave dembte bidineds mavoreld this:
Me but him dolloook, of, grow
In wile begfore that iggn: dis,
Sheir With honer, or awer's pamords of in the honourse.

CORIOLANUS:
O, be my garow an tome know nexclius makest be made;
MadddO kingstran'st hanger forceds with he form apoialop; 'twit he stray for theice
is olly spirage my man; and iftly may beide:
Say wore her; I am I fare men be
ave steenged my


## Log

- Bi-gram: 2.4716, 2.4755
- Single-head attention: 2.3899, 2.4041
- Multi-head attention, single layer: 2.0820, 2.1165
- Multi-head attention, single layer, positional encoding: 1.8575, 1.9216
- 2-layer transformer (with everything, MHA, positional encoding, layer norm): 1.7155, 1.7952